# Pipeline

## Imports

In [1]:
import os
import tensorflow as tf

from preprocessing import preprocess
from dataset import make_dataset
from model import CRNN
from util import *
import random

## Paths

In [2]:
DATA_DIR = "../../data/"
TRAIN_DIR = "train-clean-360/LibriSpeech/train-clean-360/"
VALID_DIR = "dev-clean/LibriSpeech/dev-clean/"
TEST_DIR = "test-clean/LibriSpeech/test-clean/"
NOISE_DIR = "noise/audio/"

PROCESSED_DIR = os.path.join(DATA_DIR, "processed/")
DATASET_DIR = os.path.join(DATA_DIR, "dataset/")
MODEL_DIR = "../models/"

## Parameters

In [3]:
initial_s = 10
s = 5
max_k = 10
sample_rate = 16000
create_dataset = True

## Preprocessing

In [4]:
if create_dataset:
    input_dirs = [TRAIN_DIR, VALID_DIR, TEST_DIR]
    datasets = ["train", "valid", "test"]

    for i, dataset in enumerate(datasets):
        input_dir = os.path.join(DATA_DIR, input_dirs[i])
        output_dir = os.path.join(PROCESSED_DIR, f"{dataset}/")

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        preprocess(input_dir, output_dir, initial_s)

## Creating Dataset

In [5]:
if create_dataset:
    samples = []

    for i, dataset in enumerate(datasets):
        input_dir = os.path.join(PROCESSED_DIR, f"{dataset}/")
        output_dir = os.path.join(DATASET_DIR, f"{dataset}/")

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        n_samples = make_dataset(input_dir, output_dir, max_k)
        samples.append(n_samples)

c:\Users\Marlous\anaconda3\envs\asr\lib\site-packages\pyloudnorm\normalize.py:31: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


## Add Noise for k = 0

In [6]:
if create_dataset:
    noise_files = get_files(os.path.join(DATA_DIR, NOISE_DIR), ".wav")

    for i, n_samples in enumerate(samples):
        # Select same number of noise samples as 
        # k speaker samples in each dataset
        noise_samples = random.sample(noise_files, n_samples)

        # Remove used samples
        [noise_files.remove(n) for n in noise_samples]

        # Downsample and save noise in dataset directories
        process_noise(DATASET_DIR, datasets[i], noise_samples, sample_rate)


## Train

### Data Generators

In [7]:
train_files = os.listdir(os.path.join(DATASET_DIR, "train/"))
valid_files = os.listdir(os.path.join(DATASET_DIR, "valid/"))

In [8]:
train_generator = CustomDataGenerator(os.path.join(DATASET_DIR, "train/"), train_files, dim=(500, 201), max_k=10, batch_size=32, shuffle=True)
valid_generator = CustomDataGenerator(os.path.join(DATASET_DIR, "valid/"), valid_files, dim=(500, 201), max_k=10, batch_size=1, shuffle=False)

### Model

In [9]:
model = CRNN((500, 201, 1), max_k)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 498, 199, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 496, 197, 32)      18464     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 165, 65, 32)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 163, 63, 128)      36992     
                                                                 
 conv2d_3 (Conv2D)           (None, 161, 61, 64)       73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 53, 20, 64)       0         
 2D)                                                    

### Compile

In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=tf.keras.metrics.CategoricalAccuracy(),
)

### Train

In [11]:
model.fit(train_generator, validation_data=valid_generator, epochs=10)

Epoch 1/10
4/4 [==============================] - 50s 12s/step - loss: 2.7198 - categorical_accuracy: 0.0000e+00 - val_loss: 2.4020 - val_categorical_accuracy: 0.0545
Epoch 2/10
4/4 [==============================] - 47s 12s/step - loss: 2.3838 - categorical_accuracy: 0.1328 - val_loss: 2.3812 - val_categorical_accuracy: 0.1636
Epoch 3/10
4/4 [==============================] - 44s 11s/step - loss: 2.3693 - categorical_accuracy: 0.1406 - val_loss: 2.3588 - val_categorical_accuracy: 0.1636
Epoch 4/10
4/4 [==============================] - 41s 10s/step - loss: 2.3572 - categorical_accuracy: 0.1875 - val_loss: 2.3470 - val_categorical_accuracy: 0.1273
Epoch 5/10
4/4 [==============================] - 41s 10s/step - loss: 2.3371 - categorical_accuracy: 0.1719 - val_loss: 2.3110 - val_categorical_accuracy: 0.1636
Epoch 6/10
4/4 [==============================] - 41s 10s/step - loss: 2.2994 - categorical_accuracy: 0.1719 - val_loss: 2.2814 - val_categorical_accuracy: 0.1455
Epoch 7/10
4/4 [==